In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
pip install -q langchain langchain-community langchain-openai nltk

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-bigquery 2.34.4 requires packaging<22.0dev,>=14.3, but you have packaging 24.2 which is incompatible.
jupyterlab 4.2.5 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.1.0 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
kfp 2.5.0 requires google-cloud-storage<3,>=2.2.1, but you have google-cloud-storage 1.44.0 which is incompatible.
kfp 2.5.0 requires requests-toolbelt<1,>=0.8.0, but you have requests-toolbelt 1.0.0 which is incompatible.
libpysal 4.9.2 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
thinc 8.3.2 requires numpy<2.1.0,>=2.0.0; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
ydata-profiling 4.10.0 requires scipy<1.14,>=1.4.1, but you have scipy 

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

import nltk
from tqdm import tqdm
import base64
import os
from bs4 import BeautifulSoup

In [4]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
os.environ['OPENAI_API_KEY'] = user_secrets.get_secret("OPENAI_KEY")

In [5]:
articles_df = pd.read_csv('/kaggle/input/the-batch-articles-initial/articles_html.csv')
articles_df.set_index('Url', inplace=True)
articles_df

,Title,Publication_Date,Content
Url,,,
https://www.deeplearning.ai/the-batch/art-team-sells-robots-painting-for-1-1-million/,Art team sells robot’s painting for $1.1 milli...,"Nov 11, 2024","<article class=""pb-16 bg-white""><div class=""po..."
https://www.deeplearning.ai/the-batch/swe-kit-helps-developers-build-their-own-assistants/,SWE-Kit helps developers build their own assis...,"Nov 8, 2024","<article class=""pb-16 bg-white""><div class=""po..."
https://www.deeplearning.ai/the-batch/tensions-mount-as-automation-transforms-u-s-shipping-port/,Robots On the Loading Dock Tensions mount as a...,"Nov 06, 2024","<article class=""pb-16 bg-white""><div class=""po..."
https://www.deeplearning.ai/the-batch/social-media-bots-and-the-amplification-effect/,Social Media Bots and the Amplification Effect...,"Nov 06, 2024","<article class=""pb-16 bg-white""><div class=""po..."
https://www.deeplearning.ai/the-batch/openais-mle-bench-tests-ai-coding-agents/,When Agents Train Algorithms OpenAI’s MLE-benc...,"Nov 06, 2024","<article class=""pb-16 bg-white""><div class=""po..."
...,...,...,...
https://www.deeplearning.ai/the-batch/europe-tightens-the-screws/,Europe Tightens the Screws,"Apr 17, 2019","<article class=""pb-16 bg-white""><div class=""po..."
https://www.deeplearning.ai/the-batch/vcs-bet-on-nlp/,VCs Bet on NLP,"Apr 17, 2019","<article class=""pb-16 bg-white""><div class=""po..."
https://www.deeplearning.ai/the-batch/drones-go-commercial/,Drones Go Commercial,"Apr 17, 2019","<article class=""pb-16 bg-white""><div class=""po..."


In [6]:
df = pd.read_csv('/kaggle/input/the-batch-articles-image-descriptions/image_captions.csv')
df.set_index('Unnamed: 0', inplace=True)
df

,URL,Description
Unnamed: 0,,
0,swe-kit-helps-developers-build-their-own-assis...,A futuristic scene depicting a human in a suit...
1,tensions-mount-as-automation-transforms-u-s-sh...,**Summary:** The image depicts a modern shippi...
2,social-media-bots-and-the-amplification-effect...,A cartoonish yellow robot with a red megaphone...
3,openais-mle-bench-tests-ai-coding-agents/url_h...,**Summary**: The image illustrates the MLE-ben...
4,compl-ai-study-measures-llms-compliance-with-e...,**Summary:** The image illustrates the COMPL-A...
...,...,...
1357,protein-shapes-revealed/url_https_3A_2F_2Fdl-s...,Diagram illustrating the AlphaFold model for p...
1358,nowhere-to-hide/url_https_3A_2F_2Fdl-staging-w...,"Image summary: The image features Big Ben, a p..."
1359,old-tools-for-new-synths/url_https_3A_2F_2Fdl-...,Diagram illustrating the Differentiable Digita...


#### Extract urls present

In [7]:
present_descriptions = set(df['URL'].to_numpy())
len(present_descriptions)

1362

In [8]:
images_in_dataset = set()
for dirname, _, filenames in os.walk('/kaggle/input/the-batch-articles-initial/images_clean'):
    for filename in filenames:
        images_in_dataset.add(os.path.join(dirname, filename).replace('/kaggle/input/the-batch-articles-initial/images_clean/', ''))


In [9]:
len(images_in_dataset)

1608

In [10]:
images_to_describe = images_in_dataset - present_descriptions
len(images_to_describe)

246

In [11]:
def encode_image(image_path):
    """Getting the base64 string"""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")


def image_summarize(img_base64, prompt):
    """Make image summary"""
    chat = ChatOpenAI(model="gpt-4o-mini", max_tokens=1024)

    msg = chat.invoke(
        [
            HumanMessage(
                content=[
                    {"type": "text", "text": prompt},
                    {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/jpeg;base64,{img_base64}"},
                    },
                ]
            )
        ]
    )
    return msg.content


def generate_img_summaries(img_file, context=""):
    
    # Prompt
    prompt = f"""You are an assistant tasked with summarizing images for retrieval. \
    These summaries will be embedded and used to retrieve the raw image. \
    Give a concise summary of the image that is well optimized for retrieval. \
    Consider given context: {context}"""

    # Apply to images
    base64_image = encode_image(img_file)

    return image_summarize(base64_image, prompt)


In [12]:
def generate_summary_with_context(url):
    context = ""
    
    try:
        row = articles_df.loc[f'https://www.deeplearning.ai/the-batch/{url.split("/")[0]}/']
        soup = BeautifulSoup(row['Content'], "html.parser")
        context = soup.get_text()
    except:
        context = ""

    return generate_img_summaries(f'/kaggle/input/the-batch-articles-initial/images_clean/{url}', context=context)

In [13]:
summaries_new = []
for url in tqdm(list(images_to_describe)):
    summaries_new.append([url, generate_summary_with_context(url)])

100%|██████████| 246/246 [16:32<00:00,  4.04s/it]


In [14]:
summaries_new_np = np.array(summaries_new)

df_new = pd.DataFrame(data=summaries_new_np[1:],    
            columns=['URL', 'Description'])

df_combined = pd.concat([df, df_new], ignore_index=True)
df_combined.to_csv('image_captions.csv')